In [2]:
import pandas as pd
import numpy as np
import json
import ast
import gzip

In [3]:
dtypes = {'año': str, 'id':object}
df_games = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_games.csv', dtype=dtypes)

In [4]:
df_items = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_items.csv')

In [5]:
dtypes = {'año_publicado': str}  # Cambiar 'str' al tipo de dato deseado, por ejemplo 'object'

# Cargar el archivo CSV con los tipos de datos especificados
df_review = pd.read_csv(r'D:\Users\Natalia\Desktop\csv_limpios\df_reviews.csv', dtype=dtypes)

In [46]:
df_review.isnull().sum()

user_id                   0
item_id                   0
recommend                 0
sentiment_analysis        0
año_publicado         10119
dtype: int64

In [6]:
def PlayTimeGenre(genero: str, df_games: pd.DataFrame, df_items: pd.DataFrame):
    # Verificar si el género está presente como una columna en df_games
    if genero not in df_games.columns:
        return f"No se encontró información para el género {genero}"

    # Filtrar el DataFrame de juegos para obtener los juegos que pertenecen al género específico
    juegos_genero = df_games[df_games[genero] == 1]

    if juegos_genero.empty:
        return f"No se encontró información para el género {genero}"

    # Obtener los nombres de los juegos para el género específico
    nombres_juegos_genero = juegos_genero['app_name']

    # Filtrar el DataFrame de horas jugadas para obtener las horas correspondientes a esos nombres de juegos
    horas_jugadas_genero = df_items[df_items['item_name'].isin(nombres_juegos_genero)]

    # Convertir la columna 'id' a int64 si es posible
    df_games['id'] = pd.to_numeric(df_games['id'], errors='coerce')

    # Fusionar los DataFrames después de convertir las columnas al mismo tipo
    horas_jugadas_genero = horas_jugadas_genero.merge(df_games[['id', 'año']], left_on='item_id', right_on='id')

    # Encontrar el año con más horas jugadas para el género específico
    año_mas_horas = horas_jugadas_genero.groupby('año')['playtime_forever'].sum().idxmax()

    return {f"Año de lanzamiento con más horas jugadas para el género {genero}": año_mas_horas}

In [7]:
resultado = PlayTimeGenre('Action', df_games, df_items)
print(resultado)

{'Año de lanzamiento con más horas jugadas para el género Action': '2012'}


In [8]:
def UserForGenre(genero: str, df_games: pd.DataFrame, df_items: pd.DataFrame):
    # Filtrar juegos por género
    juegos_genero = df_games[df_games[genero] == 1]
    if juegos_genero.empty:
        return f"No se encontró información para el género {genero}"

    # Obtener nombres de juegos y filtrar horas jugadas por género
    nombres_juegos_genero = juegos_genero['app_name']
    horas_jugadas_genero = df_items[df_items['item_name'].isin(nombres_juegos_genero)]

    # Fusionar DataFrames y encontrar usuario con más horas jugadas
    merged_data = horas_jugadas_genero.merge(df_games[['id', 'año']], left_on='item_id', right_on='id')
    user_mas_horas = merged_data.groupby('user_id')['playtime_forever'].sum().idxmax()

    # Calcular horas jugadas por año del usuario
    horas_por_año = merged_data[merged_data['user_id'] == user_mas_horas].groupby('año')['playtime_forever'].sum().to_dict()

    return user_mas_horas, horas_por_año

In [11]:
resultado2 = UserForGenre('Action', df_games, df_items)
print(resultado2)

('DeEggMeister', {'1994': 1.0, '1995': 80.0, '1996': 0.0, '1997': 0.0, '1998': 1504.0, '1999': 993.0, '2000': 7276.0, '2001': 295.0, '2002': 1388.0, '2003': 4364.0, '2004': 62077.0, '2005': 5118.0, '2006': 6855.0, '2007': 20012.0, '2008': 94711.0, '2009': 185133.0, '2010': 103453.0, '2011': 296094.0, '2012': 1113909.0, '2013': 434469.0, '2014': 130637.0, '2015': 202919.0, '2016': 89277.0, '2017': 87156.0})


In [23]:
def UsersRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    # Filtrar las reviews para el año dado, recomendadas y con análisis de sentimiento bueno o neutral
    filtered_reviews = df_review[(df_review['año_publicado'] == año) & 
                                 (df_review['recommend'] == True) & 
                                 (df_review['sentiment_analysis'].isin([2]))]

    # Obtener los user_ids de las reviews filtradas
    user_ids_recomendados = filtered_reviews['user_id']

    # Filtrar los items jugados por los usuarios recomendados
    items_jugados_recomendados = df_items[df_items['user_id'].isin(user_ids_recomendados)]

    # Obtener los nombres de los juegos más jugados por los usuarios recomendados
    top_games = items_jugados_recomendados['item_name'].value_counts().head(3)

    # Crear la estructura de retorno con el top 3 de juegos más jugados por usuarios recomendados
    result = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(top_games.index)]
    return result


In [30]:
def UsersRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    # Convertir la columna 'año_publicado' a tipo numérico (si es posible)
    df_review['año_publicado'] = pd.to_numeric(df_review['año_publicado'], errors='coerce')

    # Filtrar las reviews para el año dado, recomendadas y con análisis de sentimiento bueno o neutral
    filtered_reviews = df_review[(df_review['año_publicado'].notnull()) & 
                                 (df_review['año_publicado'] == año) & 
                                 (df_review['recommend'] == True) & 
                                 (df_review['sentiment_analysis']== 2) | (df_review['sentiment_analysis'] == 1)]
                                  #.isin([1, 2]))]

    # Obtener los user_ids de las reviews filtradas
    user_ids_recomendados = filtered_reviews['user_id']

    # Filtrar los items jugados por los usuarios recomendados
    items_jugados_recomendados = df_items[df_items['user_id'].isin(user_ids_recomendados)]

    # Obtener los nombres de los juegos más jugados por los usuarios recomendados
    top_games = items_jugados_recomendados['item_name'].value_counts().head(3)

    # Crear la estructura de retorno con el top 3 de juegos más jugados por usuarios recomendados
    result = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(top_games.index)]
    return result


In [31]:
resultado3 = UsersRecommend(2014, df_items, df_review)
print(resultado3)

[{'Puesto 1': 'Dota 2 Test'}, {'Puesto 2': 'Counter-Strike: Global Offensive'}, {'Puesto 3': "Garry's Mod"}]


In [19]:
#def UsersNotRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    # Convertir la columna 'año_publicado' a tipo numérico (si es posible)
    df_review['año_publicado'] = pd.to_numeric(df_review['año_publicado'], errors='coerce')

    # Filtrar las reviews para el año dado, recomendadas y con análisis de sentimiento bueno o neutral
    filtered_bad_reviews = df_review[(df_review['año_publicado'].notnull()) & 
                                 (df_review['año_publicado'] == año) & 
                                 (df_review['recommend'] == False) & 
                                 (df_review['sentiment_analysis'] == 0)]

    # Obtener los user_ids de las reviews filtradas
    user_ids_recomendados = filtered_bad_reviews['user_id']

    # Filtrar los items jugados por los usuarios recomendados
    items_jugados_recomendados = df_items[df_items['user_id'].isin(user_ids_recomendados)]

    # Obtener los nombres de los juegos más jugados por los usuarios recomendados
    less_games = items_jugados_recomendados['item_name'].value_counts().head(3)

    # Crear la estructura de retorno con el top 3 de juegos más jugados por usuarios recomendados
    result = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(less_games.index)]
    return result


In [22]:
def UsersNotRecommend(año: int, df_items: pd.DataFrame, df_review: pd.DataFrame):
    # Convertir la columna 'año_publicado' a tipo numérico (si es posible)
    df_review['año_publicado'] = pd.to_numeric(df_review['año_publicado'], errors='coerce')

    # Filtrar las reviews para el año dado, no recomendadas y con análisis de sentimiento negativo
    filtered_bad_reviews = df_review[(df_review['año_publicado'].notnull()) & 
                                     (df_review['año_publicado'] == año) & 
                                     (df_review['recommend'] == False) & 
                                     (df_review['sentiment_analysis'] == 0)]

    # Obtener los user_ids de las reviews filtradas
    user_ids_not_recommended = filtered_bad_reviews['user_id']

    # Filtrar los items jugados por los usuarios no recomendados
    items_jugados_no_recomendados = df_items[df_items['user_id'].isin(user_ids_not_recommended)]

    # Obtener los nombres de los juegos menos jugados por los usuarios no recomendados
    less_games = items_jugados_no_recomendados['item_name'].value_counts().head(3)

    # Crear la estructura de retorno con el top 3 de juegos menos jugados por usuarios no recomendados
    result = [{"Puesto " + str(i + 1): juego} for i, juego in enumerate(less_games.index)]
    return result


In [23]:
resultado5 = UsersNotRecommend(2014, df_items, df_review)
print(resultado5)

[{'Puesto 1': 'Dota 2 Test'}, {'Puesto 2': "Garry's Mod"}, {'Puesto 3': 'Counter-Strike: Global Offensive'}]


In [47]:
def sentiment_analysis(año: int, df_review: pd.DataFrame):
    # Convertir la columna 'año_publicado' a tipo numérico (si es posible)
    df_review['año_publicado'] = pd.to_numeric(df_review['año_publicado'], errors='coerce')

    # Filtrar las reviews para el año dado, excluyendo los valores nulos en 'año_publicado'
    reviews_year = df_review[(df_review['año_publicado'].notnull()) & (df_review['año_publicado'] == año)]

    # Contar la cantidad de registros por cada categoría de sentimiento para el año dado
    sentiment_counts = reviews_year['sentiment_analysis'].replace({0: 'Negative', 1: 'Neutral', 2: 'Positive'}).value_counts()

    # Crear el diccionario con la cantidad de registros por categoría de sentimiento
    result = {sentiment: sentiment_counts.get(sentiment, 0) for sentiment in ['Negative', 'Neutral', 'Positive']}
    return result


In [48]:
resultado4 = sentiment_analysis(2014, df_review)
print(resultado4)

{'Negative': 4486, 'Neutral': 4757, 'Positive': 12868}
